In [1]:
import pandas as pd

In [73]:
import numpy as np

In [5]:
import re

In [2]:
words = pd.read_csv('termdocmatrix/index_to_word.csv')

In [101]:
puncs_indexes = list()
for i in range(0, words.shape[0]):
    word = words.iloc[i, 1]
    if not (type(word) is str):
        print('At index: ' + str(i))
        print('Non str: ' + str(word))
        puncs_indexes.append(i)
        continue
        
    result = re.search(
        '[\u2000-\u206F\u0000-\u002F\u003A-\u0040\u005B-\u0060\u007B-\u007F\u0080-\u00FF\uFF00-\uFF0F\u3000-\u303F]', 
        words.iloc[i, 1]
    )
    if not (result is None):
        print('At index: ' + str(i))
        print('Result: ' + str(result))
        puncs_indexes.append(i)

At index: 28
Result: <re.Match object; span=(0, 1), match='\\'>
At index: 63
Result: <re.Match object; span=(0, 1), match='#'>
At index: 127
Result: <re.Match object; span=(2, 3), match='.'>
At index: 195
Result: <re.Match object; span=(5, 6), match='.'>
At index: 286
Result: <re.Match object; span=(1, 2), match='.'>
At index: 304
Result: <re.Match object; span=(3, 4), match='.'>
At index: 386
Result: <re.Match object; span=(1, 2), match='.'>
At index: 400
Result: <re.Match object; span=(2, 3), match='%'>
At index: 420
Result: <re.Match object; span=(2, 3), match='.'>
At index: 429
Result: <re.Match object; span=(0, 1), match='['>
At index: 517
Result: <re.Match object; span=(0, 1), match='&'>
At index: 621
Result: <re.Match object; span=(2, 3), match='.'>
At index: 707
Result: <re.Match object; span=(0, 1), match='-'>
At index: 943
Result: <re.Match object; span=(1, 2), match='.'>
At index: 1050
Result: <re.Match object; span=(1, 2), match='.'>
At index: 1079
Result: <re.Match object;

In [102]:
len(puncs_indexes)

271

In [103]:
punctuation_words = words.iloc[puncs_indexes]

In [104]:
punctuation_words.to_csv(path_or_buf='termdocmatrix/punc_words.csv', index=False)

In [105]:
pure_word_indexes = list()
puncs_indexes_set = set(puncs_indexes)
for i in range(0, words.shape[0]):
    if not (i in puncs_indexes_set):
        pure_word_indexes.append(i)

In [106]:
pure_words = words.iloc[pure_word_indexes]

In [107]:
pure_words.to_csv(path_or_buf='termdocmatrix/pure_words.csv', index=False)

In [108]:
pure_words_sorted = pure_words.sort_values('count', ascending=False)

In [109]:
pure_words_sorted.to_csv(path_or_buf='termdocmatrix/pure_words_sorted.csv', index=False)

In [110]:
a = pd.DataFrame({
    'c1': [1,2,3,4,5,6]
})

In [111]:
a

,c1
0,1
1,2
2,3
3,4
4,5
5,6
